### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


In [22]:
# setup
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from scipy.sparse import issparse

In [23]:
RNG = 42
np.random.seed(RNG)

X_tfidf = TfidfTransformer().fit_transform(X_train)
idx_sample = np.random.choice(X_tfidf.shape[0], size=5, replace=False)

def topk_sim_for_doc(i, X_mat, k=5):
    sims = linear_kernel(X_mat[i], X_mat).ravel()
    sims[i] = -1.0
    topk_idx = sims.argsort()[::-1][:k]
    return pd.DataFrame({
        "doc_idx": topk_idx,
        "similarity": sims[topk_idx],
        "label_match": [y_train[i] == y_train[j] for j in topk_idx]
    })

res_list = []
for i in idx_sample:
    df_i = topk_sim_for_doc(i, X_tfidf, k=5)
    df_i.insert(0, "query_idx", i)
    df_i.insert(2, "query_label", y_train[i])
    df_i.insert(4, "doc_label", [y_train[j] for j in df_i["doc_idx"]])
    res_list.append(df_i)

sim_tables_docs = pd.concat(res_list, ignore_index=True)
sim_tables_docs.head(20)


,query_idx,doc_idx,query_label,similarity,doc_label,label_match
0,7492,10935,4,0.783551,4,True
1,7492,7258,4,0.389983,3,False
2,7492,4971,4,0.157331,4,True
3,7492,4303,4,0.128696,6,False
4,7492,8630,4,0.109121,14,False
5,3546,2011,2,0.196060,3,False
6,3546,5665,2,0.190216,3,False
7,3546,8643,2,0.157156,3,False
8,3546,8765,2,0.156673,3,False
9,3546,10706,2,0.142067,3,False


Con TF-IDF y la similitud de coseno, los documentos más parecidos suelen compartir la misma etiqueta que el documento consulta (la columna label_match es mayormente True). Cuando aparece algún False, suele ser entre clases de temas similares, lo que tiene sentido porque comparten bastante vocabulario aunque estén en categorías distintas.

In [24]:
# punto 2

from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import classification_report, f1_score

# Usamos la misma representación TF-IDF que en el punto 1
# X_train viene de: X_train = tfidfvect.fit_transform(newsgroups_train.data)
X_train_tfidf = X_train
X_test_tfidf  = tfidfvect.transform(newsgroups_test.data)

y_train = newsgroups_train.target
y_test  = newsgroups_test.target

# Similaridad coseno (via linear_kernel, que con vectores normalizados es equivalente)
sims = linear_kernel(X_test_tfidf, X_train_tfidf)

# vecino más cercano en el conjunto de entrenamiento
nn_idx = sims.argmax(axis=1)
y_pred_nn = y_train[nn_idx]

print(classification_report(y_test, y_pred_nn, digits=3))
print("f1_macro:", f1_score(y_test, y_pred_nn, average="macro"))


              precision    recall  f1-score   support

           0      0.366     0.508     0.425       319
           1      0.543     0.483     0.512       389
           2      0.506     0.457     0.480       394
           3      0.515     0.520     0.518       392
           4      0.535     0.499     0.516       385
           5      0.701     0.592     0.642       395
           6      0.629     0.462     0.533       390
           7      0.406     0.576     0.476       396
           8      0.635     0.515     0.569       398
           9      0.645     0.537     0.586       397
          10      0.748     0.722     0.735       399
          11      0.552     0.588     0.570       396
          12      0.531     0.328     0.406       393
          13      0.652     0.492     0.561       396
          14      0.644     0.505     0.566       394
          15      0.451     0.580     0.508       398
          16      0.454     0.470     0.462       364
          17      0.363    

El clasificador por prototipos (tomar la clase del documento de entrenamiento más similar en TF-IDF + coseno) logra un f1_macro de ~0.50. Es mejor que adivinar al azar, pero bastante peor que Naïve Bayes: algunas clases tienen buen f1, pero en otras el desempeño es bajo. Al depender de un solo vecino, el modelo es muy sensible a ruido o documentos ambiguos.

In [25]:
# punto 3 - versión corregida y más liviana

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import classification_report, f1_score

RNG = 42  # por si lo usás en algún otro lado

# usamos directamente los textos crudos de train y test
X_train_text = newsgroups_train.data
y_train = newsgroups_train.target
X_test_text = newsgroups_test.data
y_test = newsgroups_test.target

# pipelines: TEXTO -> TFIDF (no negativo) -> NB
pipe_mnb = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB()),
])

pipe_cnb = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", ComplementNB()),
])

# grilla reducida: solo hiperparámetros del clasificador
param_mnb = {
    "clf__alpha": [0.1, 0.5, 1.0],
    "clf__fit_prior": [True, False],
}

param_cnb = {
    "clf__alpha": [0.1, 0.5, 1.0],
    "clf__fit_prior": [True, False],
}

# Grid search SOLO en el conjunto de train
gs_mnb = GridSearchCV(
    pipe_mnb,
    param_grid=param_mnb,
    scoring="f1_macro",
    cv=3,        # menos folds para que termine rápido
    n_jobs=-1,
    verbose=1,   # para ver progreso y confirmar que corre
)

gs_cnb = GridSearchCV(
    pipe_cnb,
    param_grid=param_cnb,
    scoring="f1_macro",
    cv=3,
    n_jobs=-1,
    verbose=1,
)

gs_mnb.fit(X_train_text, y_train)
gs_cnb.fit(X_train_text, y_train)

print("Best params MNB:", gs_mnb.best_params_)
print("Best CV f1_macro MNB:", round(gs_mnb.best_score_, 4))

print("Best params CNB:", gs_cnb.best_params_)
print("Best CV f1_macro CNB:", round(gs_cnb.best_score_, 4))

# evaluación FINAL en el set de test original
best_mnb = gs_mnb.best_estimator_
best_cnb = gs_cnb.best_estimator_

pred_mnb = best_mnb.predict(X_test_text)
pred_cnb = best_cnb.predict(X_test_text)

score_mnb = f1_score(y_test, pred_mnb, average="macro")
score_cnb = f1_score(y_test, pred_cnb, average="macro")

print("\nTest f1_macro MNB:", round(score_mnb, 4))
print("Test f1_macro CNB:", round(score_cnb, 4))
print("\nMNB report:\n", classification_report(y_test, pred_mnb, digits=3))
print("\nCNB report:\n", classification_report(y_test, pred_cnb, digits=3))
print("winner:", "MNB" if score_mnb >= score_cnb else "CNB")


Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best params MNB: {'clf__alpha': 0.1, 'clf__fit_prior': False}
Best CV f1_macro MNB: 0.7204
Best params CNB: {'clf__alpha': 0.1, 'clf__fit_prior': True}
Best CV f1_macro CNB: 0.7534

Test f1_macro MNB: 0.6664
Test f1_macro CNB: 0.6954

MNB report:
               precision    recall  f1-score   support

           0      0.263     0.292     0.277       319
           1      0.680     0.699     0.689       389
           2      0.708     0.523     0.601       394
           3      0.605     0.740     0.666       392
           4      0.767     0.701     0.733       385
           5      0.829     0.747     0.786       395
           6      0.850     0.728     0.785       390
           7      0.814     0.750     0.781       396
           8      0.828     0.726     0.774       398
           9      0.936     0.811     0.869       397
          10      0.919     0.910    

Con las pipelines TF-IDF + Naïve Bayes, el desempeño mejora claramente. En validación, el mejor MultinomialNB queda alrededor de f1_macro ≈ 0.72 y el mejor ComplementNB cerca de 0.75. En el conjunto de test, ComplementNB sigue siendo el mejor, con f1_macro ≈ 0.70 y una accuracy global cercana a 0.71. La mayoría de las clases tienen f1 entre 0.6 y 0.8, por lo que este modelo es bastante más estable y preciso que el clasificador por prototipos.

In [26]:
TERM_SEEDS = ["inflation", "prices", "market", "policy", "growth"]

def _find_vectorizer_in_globals():
    cand = []
    for name, obj in globals().items():
        if hasattr(obj, "get_feature_names_out"):
            try:
                _ = obj.get_feature_names_out()
                cand.append(obj)
            except Exception:
                pass
    return cand[0] if cand else None

def term_similarity(X_source, term_seeds=None, topk=5):
    if issparse(X_source):
        X_counts_or_tfidf = X_source
        tf = TfidfTransformer()
        X_tfidf = tf.fit_transform(X_counts_or_tfidf)
        TD = X_tfidf.T
        vec = _find_vectorizer_in_globals()
        if vec is not None:
            try:
                names = np.array(vec.get_feature_names_out())
            except Exception:
                names = np.array([f"f{i}" for i in range(TD.shape[0])])
        else:
            names = np.array([f"f{i}" for i in range(TD.shape[0])])
        col_freq = np.asarray(X_counts_or_tfidf.sum(axis=0)).ravel()
        default_seeds = names[np.argsort(col_freq)[::-1][:5]].tolist()
    else:
        cnt = CountVectorizer()
        X_cnt = cnt.fit_transform(X_source)
        names = np.array(cnt.get_feature_names_out())
        col_freq = np.asarray(X_cnt.sum(axis=0)).ravel()
        tfidf = TfidfVectorizer()
        X_tfidf = tfidf.fit_transform(X_source)
        TD = X_tfidf.T
        default_seeds = names[np.argsort(col_freq)[::-1][:5]].tolist()

    seeds = term_seeds if term_seeds else default_seeds

    name_to_idx = {n:i for i,n in enumerate(names)}
    out = []
    for s in seeds:
        if isinstance(s, int):
            i = s if 0 <= s < TD.shape[0] else None
        else:
            i = name_to_idx.get(s, None)
        if i is None:
            continue
        sims = cosine_similarity(TD[i], TD).ravel()
        sims[i] = -1.0
        idx = sims.argsort()[::-1][:topk]
        terms = names[idx].tolist()
        out.append(pd.DataFrame({"query_term": names[i], "term": terms, "similarity": sims[idx]}))
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

sim_tables_terms = term_similarity(X_train, term_seeds=None, topk=5)
sim_tables_terms.head(20)

sim_tables_terms = term_similarity(X_train, term_seeds=TERM_SEEDS, topk=5)
sim_tables_terms.head(20)

,query_term,term,similarity
0,inflation,fv,0.799885
1,inflation,measly,0.526053
2,inflation,speculators,0.480622
3,inflation,salary,0.389040
4,inflation,mortgage,0.355063
5,prices,edmund,0.294779
6,prices,thatn,0.238674
7,prices,authorised,0.231423
8,prices,guides,0.222430
9,prices,mailorder,0.205158


Al buscar palabras similares a inflation, prices, market, policy y growth aparecen términos que coocurren seguido con ellas, como salary, mortgage o unfavorable, mezclados con nombres propios o typos (edmund, stragegy, etc.). Esto muestra que la similaridad está basada en coocurrencia en los documentos, no solo en sinónimos; si quisiéramos interpretaciones más claras habría que filtrar o limpiar mejor el vocabulario.